In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import pandas as pd
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.offline as pyo
import plotly.express as px
pyo.init_notebook_mode(connected=True)
import utils as u

In [16]:
"""
Plain old dataset:
- BOX RMSRE (varying epoch sizes)
- CDF BUDGET (epoch_size=7)
- AVG BUDGET for 3 advs (epoc_size 7)

Enhanced with impressions:
- BOX RMSRE (varying epoch sizes)
- CDF BUDGET (epoch_size=7)
- AVG BUDGET for 3 advs (epoc_size 7)
"""

'\nPlain old dataset:\n- BOX RMSRE (varying epoch sizes)\n- CDF BUDGET (epoch_size=7)\n- AVG BUDGET for 3 advs (epoc_size 7)\n\nEnhanced with impressions:\n- BOX RMSRE (varying epoch sizes)\n- CDF BUDGET (epoch_size=7)\n- AVG BUDGET for 3 advs (epoc_size 7)\n'

In [17]:
"""
These destinations are selected based on their query pool sizes, and how representative
they are of the query pool distributions (min 1 query, mean 8.24 queries, max 93 queries)

9FF550C0B17A3C493378CB6E2DEEE6E4 has the largest query pool (93),
482E924BCFAA8562F60BF0BE53786C24 has a query pool close to the mean (8),
9D883ED995259EA3E715122397FE0EF9 has query pool of size 2
"""
selected_destinations = ["9FF55", "482E9", "9D883"]

In [18]:

def extra_df_prep(df: pd.DataFrame) -> pd.DataFrame:
    df = df.assign(
        destination=df.apply(
            lambda row: row.destination[0:5],
            axis=1,
        )
    )
    destinations_to_show = 3
    global selected_destinations
    if selected_destinations is None:
        selected_destinations = df.destination.sample(frac=1).unique()[:destinations_to_show]
    return df[df.destination.isin(selected_destinations)]


In [19]:
# unaugmented path
path = "ray/criteo/one_tenth_augment/bias_varying_epoch_size"

In [20]:
# u.plot_rmsre_boxes(path, log_y=True, category_orders={"num_days_per_epoch": [1, 7, 14, 21, 30, 60, 90]})

In [21]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [22]:

u.plot_budget_consumption_lines(
    path,
    epoch_size=7,
    by_destination=True,
    extra_df_prep=extra_df_prep
)

In [23]:
# augmented path
path = "ray/criteo/one_tenth_augment/augmented_bias_varying_epoch_size"

In [24]:
# u.plot_rmsre_boxes(path, log_y=True, category_orders={"num_days_per_epoch": [1, 7, 14, 21, 30, 60, 90]})

In [25]:
# u.plot_budget_consumption_cdf(path, epoch_size=7)

In [26]:

u.plot_budget_consumption_lines(
    path,
    epoch_size=7,
    by_destination=True,
    extra_df_prep=extra_df_prep
)